# Samalas 1258 ENSO Analysis

This notebook consolidates helper functions and analysis workflows for investigating how the 1258 Samalas eruption impacts ENSO and surface temperatures across different eruption seasons.


## Prerequisites

This notebook expects the CESM Last Millennium Ensemble surface temperature data to be available on your system.
Update the path constants below to match your local NetCDF files.

Install the Python packages listed in `requirements.txt` using `pip install -r requirements.txt` before executing the notebook.


In [ ]:
# ---- Imports and basic configuration ----

# Standard library
from datetime import datetime
from itertools import product

# Scientific computing libraries
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats
import cftime
from cftime import DatetimeNoLeap

# Progress bar and plotting helpers
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as mcolors

# Mapping utilities
from collections import OrderedDict, namedtuple
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely.geometry import LinearRing, Polygon
from netCDF4 import Dataset as nc
from netCDF4 import num2date

# Utility to capture the current timestamp (useful for file naming)
def get_current_datetime():
    now = datetime.now()
    return now.strftime('%D'), now.strftime('%H:%M:%S')

# ---- File paths and naming conventions ----
# Update these strings to match your local directory structure.
FILEPATH = '/glade/derecho/scratch/calipfleger/ilme/'
BASEDIR = '/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/'
BASEDIR_OCN = '/glade/derecho/scratch/samantha/iLME_seasvolc/CESM-CAM5-LME/atm/proc/tseries/monthly/'
FILE_PREFIX = '.cam.h0.'


## ENSO index helpers

The functions below compute area-weighted means over standard ENSO regions.
They reproduce helper routines originally defined in the `V1_1x_Sam.ipynb` notebook.


In [ ]:
# ---- Nino index calculations ----
import numpy as np
import xarray as xr

def _area_weighted_mean(data, lat_name='lat', lon_name='lon'):
    # Compute cosine-latitude weighted mean of a DataArray
    weights = np.cos(np.deg2rad(data[lat_name]))
    return data.weighted(weights).mean(dim=[lat_name, lon_name])

def nino_region(data, lat_bounds, lon_bounds, lat_name='lat', lon_name='lon'):
    # Slice a DataArray to the specified lat/lon box and take the weighted mean
    subset = data.sel({lat_name: slice(*lat_bounds), lon_name: slice(*lon_bounds)})
    return _area_weighted_mean(subset, lat_name=lat_name, lon_name=lon_name)

def nino34(data, lat_name='lat', lon_name='lon'):
    # Compute the Niño 3.4 index (5S–5N, 190E–240E)
    return nino_region(data, (-5,5), (190,240), lat_name, lon_name)

def nino3(data, lat_name='lat', lon_name='lon'):
    # Compute the Niño 3 index (5S–5N, 210E–270E)
    return nino_region(data, (-5,5), (210,270), lat_name, lon_name)

def nino4(data, lat_name='lat', lon_name='lon'):
    # Compute the Niño 4 index (5S–5N, 160E–210E)
    return nino_region(data, (-5,5), (160,210), lat_name, lon_name)

# Legacy helpers from the original notebook
def calculate_nino34(data):
    n34 = data.sel(lat=slice(-5,5), lon=slice(190,240))
    weights = np.cos(np.deg2rad(n34.lat))
    return n34.weighted(weights).mean(dim=['lat','lon'])

def season_nino34(m):
    DJF = m.sel(time=m.time.dt.season == 'DJF')
    JJA = m.sel(time=m.time.dt.season == 'JJA')
    return DJF, JJA


## Eruption analysis workflow

The following utilities load surface temperature ensembles, compute anomalies relative to a control run,
classify the pre-eruption ENSO phase, and compare post-eruption responses across different eruption months.


In [ ]:
# ---- Analysis utilities ----
from itertools import combinations
from typing import Iterable, Dict, Tuple
import numpy as np
import xarray as xr
from scipy import stats

from nino_indices import nino34
from samalas_setup import FILEPATH, VAR_NAME

def load_ts_ensemble(onset, members, variable='TS'):
    # Load a 10-member ensemble of surface temperature for a given eruption onset month
    paths = [f"{FILEPATH}{onset}/{variable}_m{m:02d}.nc" for m in members]
    datasets = [xr.open_dataarray(p) for p in paths]
    return xr.concat(datasets, dim='member')

def compute_ts_anomaly(exp, control):
    # Calculate anomalies relative to a control ensemble
    control_clim = control.groupby('time.month').mean('time')
    return exp.groupby('time.month') - control_clim

def classify_pre_eruption_phase(sst, months=12, threshold=1.0, eruption_index=None):
    # Label each ensemble member by its mean Niño 3.4 value during the months leading up to the eruption
    if eruption_index is None:
        eruption_index = months
    n34 = nino34(sst)
    pre = n34.isel(time=slice(eruption_index - months, eruption_index))
    mean = pre.mean('time')
    std = pre.std('time')
    phase = xr.full_like(mean, 'Neutral', dtype=object)
    phase = xr.where(mean > threshold * std, 'El Nino', phase)
    phase = xr.where(mean < -threshold * std, 'La Nina', phase)
    return phase

def composite_post_eruption(ts_anom, phases, months=24, eruption_index=None):
    # Average post-eruption anomalies according to the initial ENSO phase
    if eruption_index is None:
        eruption_index = 0
    post = ts_anom.isel(time=slice(eruption_index, eruption_index + months))
    results = {}
    for phase in np.unique(phases):
        mask = phases == phase
        results[phase] = post.sel(member=mask).mean('member')
    return results

def _global_mean(data, lat_name='lat', lon_name='lon'):
    # Compute cosine-weighted global mean
    weights = np.cos(np.deg2rad(data[lat_name]))
    return data.weighted(weights).mean(dim=[lat_name, lon_name])

def _nino34_anomaly(ts, control=None, pre_months=12, eruption_index=None):
    # Return Niño 3.4 anomalies, optionally relative to a control run
    if eruption_index is None:
        eruption_index = pre_months
    if control is not None:
        ts_anom = compute_ts_anomaly(ts, control)
    else:
        base = ts.isel(time=slice(eruption_index - pre_months, eruption_index)).mean('time')
        ts_anom = ts - base
    return nino34(ts_anom)

def ttest_onset_differences(onsets=VAR_NAME, members=range(1,11), variable='TS',
                            pre_months=12, post_months=24, control=None, eruption_index=None):
    # Perform pairwise t-tests on global-mean TS and Niño 3.4 anomalies across eruption months
    if eruption_index is None:
        eruption_index = pre_months
    ts_means = {}
    n34_anoms = {}
    for onset in onsets:
        ts = load_ts_ensemble(onset, members, variable=variable)
        if control is not None:
            ts_anom = compute_ts_anomaly(ts, control)
        else:
            base = ts.isel(time=slice(eruption_index - pre_months, eruption_index)).mean('time')
            ts_anom = ts - base
        ts_mean = _global_mean(ts_anom).isel(time=slice(eruption_index, eruption_index + post_months))
        n34 = _nino34_anomaly(ts, control, pre_months, eruption_index).isel(time=slice(eruption_index, eruption_index + post_months))
        ts_means[onset] = ts_mean
        n34_anoms[onset] = n34
    results = {}
    for a, b in combinations(onsets, 2):
        ts_t, ts_p = stats.ttest_ind(ts_means[a], ts_means[b], axis=0, equal_var=False)
        n34_t, n34_p = stats.ttest_ind(n34_anoms[a], n34_anoms[b], axis=0, equal_var=False)
        results[(a, b)] = xr.Dataset({'ts_t': ('time', ts_t), 'ts_p': ('time', ts_p),
                                      'n34_t': ('time', n34_t), 'n34_p': ('time', n34_p)},
                                     coords={'time': ts_means[a].time})
    return results
